In [1]:
import pandas as pd
import random
import mpu
import json
import requests

In [49]:
data = pd.read_csv("loan_cross_sell.csv")

In [50]:
data.head()

,customer_id,loan_availed,gender,marital_status,dependents,education,self_employed,applicant_income,coapplicant_income,loan_amt,loan_amt_term,credit_history,owns_house,owns_vehicle,dob,customer_to_bank
0,C1000,Business Growth Loan,Male,unmarried,0,Not Graduate,No,2573383,522906,2403229,43,0,No,Yes,09/12/1971,09/25/2012
1,C1001,Two wheeler Loan,Male,unmarried,3+,Graduate,Yes,383674,448652,560608,18,1,Yes,No,06/18/1970,04/23/2016
2,C1002,Two wheeler Loan,Male,married,1,Graduate,Yes,479774,444645,1938408,19,1,No,No,09/25/1972,02/01/2010
3,C1003,Personal Loan,Female,unmarried,1,Not Graduate,Yes,745407,444115,236050,17,0,Yes,No,07/15/1980,06/20/2013
4,C1004,Personal Loan,Female,married,1,Graduate,Yes,554936,527709,170099,22,1,Yes,Yes,06/12/1975,10/31/2014


In [51]:
# adding a few addresses with places

data['address']=""

list_1= ["Fraser Town,Bengaluru" , "J.C.Nagar,Bengaluru","Air Force Hospital,Bengaluru","Agram,Bengaluru",
         "Madhavan Park,Bengaluru","Vimanapura,Bengaluru","Yeswanthpura,Bengaluru","Hulsur Bazaar,Bengaluru" ]

add_list=[]
for i in range(len(data)):
    add_list.append(random.choice(list_1))
    

data['address']=add_list
    

In [52]:
# adding few occupations based 
data.drop('education',axis=1,inplace=True)


In [53]:
data['applicant_income'].min()

115757

In [54]:
data['applicant_income'].max()

3215719

In [55]:
data['education'] = ""

edu_lis=[]

for i in range(len(data)):

    if data['applicant_income'][i]<400000:
        edu_lis.append('Not Graduate')
    
    else:
        edu_lis.append('Graduate')
        
data['education']=edu_lis

In [56]:
# adding occupation 

occ_list=[]
self_emp=[]

data['occupation']=""

for i in range(len(data)):
    
    if data['loan_availed'][i]=="Business Growth Loan":
        occ_list.append("Business")
        self_emp.append("Yes")
        
    elif data['education'][i]=="Graduate":
        occ_list.append(random.choice(['Engineer','Doctor','Lawyer']))
        
        if occ_list[i]=="Engineer":
            self_emp.append("No")
        else:
            self_emp.append(random.choice(["Yes","No"]))
            
        
    else:
        occ_list.append(random.choice(['Plumber','Welder','BrickLayer']))
        self_emp.append("Yes")

data['occupation'] = occ_list
data['self_employed']=self_emp

In [57]:
data.head()

,customer_id,loan_availed,gender,marital_status,dependents,self_employed,applicant_income,coapplicant_income,loan_amt,loan_amt_term,credit_history,owns_house,owns_vehicle,dob,customer_to_bank,address,education,occupation
0,C1000,Business Growth Loan,Male,unmarried,0,Yes,2573383,522906,2403229,43,0,No,Yes,09/12/1971,09/25/2012,"Yeswanthpura,Bengaluru",Graduate,Business
1,C1001,Two wheeler Loan,Male,unmarried,3+,Yes,383674,448652,560608,18,1,Yes,No,06/18/1970,04/23/2016,"Fraser Town,Bengaluru",Not Graduate,Plumber
2,C1002,Two wheeler Loan,Male,married,1,Yes,479774,444645,1938408,19,1,No,No,09/25/1972,02/01/2010,"Yeswanthpura,Bengaluru",Graduate,Lawyer
3,C1003,Personal Loan,Female,unmarried,1,No,745407,444115,236050,17,0,Yes,No,07/15/1980,06/20/2013,"J.C.Nagar,Bengaluru",Graduate,Doctor
4,C1004,Personal Loan,Female,married,1,No,554936,527709,170099,22,1,Yes,Yes,06/12/1975,10/31/2014,"Agram,Bengaluru",Graduate,Doctor


In [58]:
data.to_csv("loan_cross_sell.csv",index=False)

In [11]:
# passing through apis

def zipcode_distance_retrieval(self,col,country):
    
        list_dist=[]
        
            
        address = col

        address = address.split(",")

        """USAGE - get zipcode of place("Whitefield")"""

        place = address[0].title()
        city  = address[1].title()
        place_dict = {  "adminName2": city,
                         "placeName": place}

        q = 'https://parseapi.back4app.com/classes/Worldzipcode_'+country+'?limit=1&excludeKeys=accuracy,adminCode1,adminCode2&where=' + json.dumps(
          place_dict)
        query = json.dumps(q)

        response = requests.get(json.loads(query),

                                headers={
                                    'X-Parse-Application-Id': 'GSNkm3CSoxzgwo47wCdufbaDFA3SvgqRh9hJ82mt',
                                    'X-Parse-REST-API-Key':   'brIpMJbLmH565ZiY1ODhpLX08cO4N9jdqiedqAHY'
                                })

        data_res = response.json()


        if response.status_code!=404:

            result  = data_res['results'][0]['geoPosition']
            zipcode = data_res['results'][0]['postalCode']

            zip_1 = (result['latitude'],result['longitude'])

            # fix position of zip code 2

            zip_2 = (12.9698,77.7500)

            dist =round(mpu.haversine_distance(zip_1,zip_2),2)

            #print(dist)

            
            return dist

        else:
            return 0


In [12]:
dict_add={}

address = list(data['address'].unique())

for add in address:
    dict_add[add]=zipcode_distance_retrieval(add,"IN")

In [13]:
dict_add

{'Yeswanthpura,Bengaluru': 12.8,
 'Hulsur Bazaar,Bengaluru': 18.11,
 'Air Force Hospital,Bengaluru': 18.11,
 'Madhavan Park,Bengaluru': 19.82,
 'Vimanapura,Bengaluru': 19.82,
 'J.C.Nagar,Bengaluru': 18.11,
 'Agram,Bengaluru': 18.11,
 'Fraser Town,Bengaluru': 18.11}

In [14]:
list_add = dict_add.keys()

In [15]:
def address_distance(data,dict_add,col):
    
    list_add = dict_add.keys()
    
    dist=[]
    
    for i in range(len(data)):
        
        if int(dict_add[data[col][i]]) < 15:
            dist.append(1)
        else:
            dist.append(0)
        
    data.drop(col,axis=1,inplace=True)
    
    
    data["address_near_by_bank"] =dist
    
    return data    

In [16]:
data = address_distance(data,dict_add,"address")

In [17]:
data.head()

,customer_id,loan_availed,gender,marital_status,dependents,self_employed,applicant_income,coapplicant_income,loan_amt,loan_amt_term,credit_history,owns_house,owns_vehicle,dob,customer_to_bank,education,occupation,address_near_by_bank
0,C1000,Business Growth Loan,Male,unmarried,0,Yes,2573383,522906,2403229,43,0,No,Yes,09/12/1971,09/25/2012,Graduate,Business,1
1,C1001,Two wheeler Loan,Male,unmarried,3+,Yes,383674,448652,560608,18,1,Yes,No,06/18/1970,04/23/2016,Not Graduate,BrickLayer,0
2,C1002,Two wheeler Loan,Male,married,1,Yes,479774,444645,1938408,19,1,No,No,09/25/1972,02/01/2010,Graduate,Doctor,1
3,C1003,Personal Loan,Female,unmarried,1,No,745407,444115,236050,17,0,Yes,No,07/15/1980,06/20/2013,Graduate,Engineer,0
4,C1004,Personal Loan,Female,married,1,No,554936,527709,170099,22,1,Yes,Yes,06/12/1975,10/31/2014,Graduate,Lawyer,0


In [18]:
data['address_near_by_bank'].value_counts()

0    96757
1    13743
Name: address_near_by_bank, dtype: int64

In [43]:
# occupation -> yaml restrictions and scores update

import yaml

def read_yaml_filename(filename,attribute):

    with open(filename) as file:
        decisions = yaml.load(file, Loader=yaml.FullLoader)

    return decisions[attribute]



In [46]:
file_yaml    = read_yaml_filename("temp.yml","occupation")

In [47]:
len(file_yaml)

3

In [48]:
file_yaml

{'lawyer': [{'attribute': 'income',
   'type': 'non-list',
   'required_values': {'min': False, 'max': 500000}},
  {'attribute': 'self_employed', 'type': 'list', 'values': ['Yes']},
  {'attribute': 'score',
   'type': 'non-list',
   'required_values': {'min': 0.6, 'max': 0.8}}],
 'doctor': [{'attribute': 'income',
   'type': 'non-list',
   'required_values': {'min': False, 'max': 700000}},
  {'attribute': 'self_employed', 'type': 'list', 'values': ['Yes']},
  {'attribute': 'score',
   'type': 'non-list',
   'required_values': {'min': 0.5, 'max': 0.7}}],
 'business': [{'attribute': 'income',
   'type': 'non-list',
   'required_values': {'min': False, 'max': 500000}},
  {'attribute': 'score',
   'type': 'non-list',
   'required_values': {'min': 0.7, 'max': 0.9}}]}

In [ ]:
for i in range(len(file_yaml)):
    
    
    